In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [3]:
sources = {
    "litoral": {
        "violencia_narco": "https://www.ellitoral.com/tag/violencia-narco",
        "narcotrafico": "https://www.ellitoral.com/tag/narcotrafico",
    },
    "aire": {
        "droga": "https://www.airedesantafe.com.ar/droga-a848/",
        "narcotrafico": "https://www.airedesantafe.com.ar/narcotrafico-a534",
        "cocaina": "https://www.airedesantafe.com.ar/cocaina-a1378",
        "drogas": "https://www.airedesantafe.com.ar/drogas-a4787",
    },
    "rosario12": {
        "general": "https://www.pagina12.com.ar/suplementos/rosario12/{date}",
    },
}

In [4]:
def scrape_links_and_titles_litoral(url):
    # Send a GET request to the webpage
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the webpage content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the div with the specific class
        div = soup.find('div', class_='styles_temas-container__2moeC')

        if div:
            # Prepare a list to store the data
            data = []

            # Find all <a> tags within the div
            links = div.find_all('a', href=True)

            for link in links:
                href = url + link['href']  # Prepend the prefix to the href attribute

                # Find the <h1> inside the <a> tag (if it exists)
                h1 = link.find('h1')
                title = h1.get_text(strip=True) if h1 else None  # Extract text or use None if not found

                # Append the link and title to the data list
                data.append({'link': href, 'title': title})

            # Create a DataFrame from the data list
            df = pd.DataFrame(data)

            return df
        else:
            print("No div with the specified class was found.")
            return pd.DataFrame()  # Return an empty DataFrame
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame



NameError: name 'scrape_links_and_titles' is not defined

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_with_multiple_clicks_litoral(sources):
    
    driver = webdriver.Chrome()  # Initialize the Chrome WebDriver

    # List to store all scraped data
    data = []

    # Extract URLs and tags
    urls = list(sources["litoral"].values())
    tags = list(sources["litoral"].keys())

    try:
        # Iterate through each URL and tag
        for url, tag in zip(urls, tags):
            try:
                # Load the webpage
                driver.get(url)

                # Wait for the page to load initially
                time.sleep(5)

                try:
                    cancel_button = driver.find_element(By.ID, "onesignal-slidedown-cancel-button")
                    cancel_button.click()
                    time.sleep(2)  # Wait some time after clicking the cancel button
                except NoSuchElementException:
                    print(f"The 'onesignal-slidedown-cancel-button' button not found on {url}.")
                    pass                

                # Click the button until it no longer exists
                while True:
                    try:                        
                        # Find and click the button
                        button = driver.find_element(By.CLASS_NAME, "styles_btn__Y8WwT")
                        
                        button.click()
                        time.sleep(5)  # Wait for new content to load
                    # Scroll down the page
                        actions = ActionChains(driver)
                        actions.send_keys(Keys.END).perform()  # Scroll to the bottom of the page
                        time.sleep(2)  # Wait for the page to fully load after scrolling
                        try:
                            cancel_button = driver.find_element(By.ID, "onesignal-slidedown-cancel-button")
                            cancel_button.click()
                            time.sleep(2)  # Wait some time after clicking the cancel button
                        except NoSuchElementException:
                            print(f"The 'onesignal-slidedown-cancel-button' button not found on {url}.")
                            pass                
                            
                    except NoSuchElementException as e:
                        print(f"NoSuchElementException: Button not found on {url}. Error: {e}")
                        break
                    except ElementNotInteractableException as e:
                        print(f"ElementNotInteractableException: Button found but not interactable on {url}. Error: {e}")
                        break
                    except ElementClickInterceptedException as e:
                        print(f"ElementClickInterceptedException: Button click intercepted on {url}. Error: {e}")
                        break

                # Extract the final page source
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Scrape the content using BeautifulSoup
                div = soup.find('div', class_='styles_temas-container__2moeC')

                if div:
                    # Find all <a> tags within the div
                    links = div.find_all('a', href=True)

                    for link in links:
                        href = url + link['href']  # Prepend the URL prefix to the href attribute

                        # Find the <h1> inside the <a> tag (if it exists)
                        h1 = link.find('h1')
                        title = h1.get_text(strip=True) if h1 else None  # Extract text or use None if not found

                        # Append the link, title, and additional info to the data list
                        data.append({'Link': href, 'Title': title, 'Tag': tag, 'Media': 'ellitoral'})

                else:
                    print(f"No div with the specified class found on {url}.")

            except Exception as e:
                print(f"Error processing {url}: {e}")

    finally:
        # Close the WebDriver
        driver.quit()

    # Create a DataFrame from the data list
    return pd.DataFrame(data)

In [20]:
df_litoral = scrape_with_multiple_clicks_litoral(sources)

The 'onesignal-slidedown-cancel-button' button not found on https://www.ellitoral.com/tag/violencia-narco.
ElementClickInterceptedException: Button click intercepted on https://www.ellitoral.com/tag/violencia-narco. Error: Message: element click intercepted: Element <button class="styles_btn__Y8WwT undefined">...</button> is not clickable at point (455, 846). Other element would receive the click: <iframe id="google_ads_iframe_/21877992475/ellitoral/interior_zocalo_0" name="google_ads_iframe_/21877992475/ellitoral/interior_zocalo_0" title="3rd party ad content" width="728" height="90" scrolling="no" marginwidth="0" marginheight="0" frameborder="0" aria-label="Advertisement" tabindex="0" allow="private-state-token-redemption;attribution-reporting" data-load-complete="true" data-google-container-id="3" style="border: 0px; vertical-align: bottom; background: none 0px 0px repeat scroll white; margin: auto;"></iframe>
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0

In [82]:
df_narcotrafico = scrape_with_multiple_clicks_litoral(sources)

ElementClickInterceptedException: Button click intercepted on https://www.ellitoral.com/tag/narcotrafico. Error: Message: element click intercepted: Element <button class="styles_btn__Y8WwT undefined">...</button> is not clickable at point (455, 169). Other element would receive the click: <div id="onesignal-slidedown-dialog" class="onesignal-slidedown-dialog">...</div>
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF67306FB05+28789]
	(No symbol) [0x00007FF672FD86E0]
	(No symbol) [0x00007FF672E7592A]
	(No symbol) [0x00007FF672ED0FDE]
	(No symbol) [0x00007FF672ECEA7C]
	(No symbol) [0x00007FF672ECBC46]
	(No symbol) [0x00007FF672ECABA1]
	(No symbol) [0x00007FF672EBCD60]
	(No symbol) [0x00007FF672EEF43A]
	(No symbol) [0x00007FF672EBC5B6]
	(No symbol) [0x00007FF672EEF650]
	(No symbol) [0x00007FF672F0F654]
	(No symbol) [0x00007FF672EEF1E3]
	(No symbol) [0x00007FF672EBA938]
	(No symbol) [0x00007FF672EBBAA1]
	GetHandleVerifier [0x00007FF6733A933D+3410093]
	GetH

In [89]:
df_ellitoral = pd.concat([df_litoral, df_narcotrafico], ignore_index=True)
df_ellitoral

,Link,Title,Tag,Media
0,https://www.ellitoral.com/tag/violencia-narco/...,Asesinaron a tiros a un alcalde y otras tres p...,violencia_narco,ellitoral
1,https://www.ellitoral.com/tag/violencia-narco/...,A la cárcel: narcos barriales deberán pasar un...,violencia_narco,ellitoral
2,https://www.ellitoral.com/tag/violencia-narco/...,"Acribillaron a Benito Aguas Atlahua, diputado ...",violencia_narco,ellitoral
3,https://www.ellitoral.com/tag/violencia-narco/...,"""Esto nunca va a terminar"": volvieron a amenaz...",violencia_narco,ellitoral
4,https://www.ellitoral.com/tag/violencia-narco/...,Detuvieron en Buenos Aires a prófugo por el cr...,violencia_narco,ellitoral
...,...,...,...,...
332,https://www.ellitoral.com/tag/narcotrafico/suc...,Detuvieron en Villa Oculta a un narco que estu...,narcotrafico,ellitoral
333,https://www.ellitoral.com/tag/narcotrafico/suc...,Múltiples allanamientos por microtráfico en la...,narcotrafico,ellitoral
334,https://www.ellitoral.com/tag/narcotrafico/suc...,Detuvieron a siete personas vinculadas a una b...,narcotrafico,ellitoral
335,https://www.ellitoral.com/tag/narcotrafico/pol...,"Firme pero sin estridencias, Pullaro aseguró q...",narcotrafico,ellitoral


In [1]:
df_ellitoral["Link"][0]

NameError: name 'df_ellitoral' is not defined

In [ ]:
def content_extract(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        # Decode the content as utf-8 explicitly
        html_content = response.content.decode('utf-8', errors='ignore')
        soup = BeautifulSoup(html_content, 'html.parser')
        content = soup.findAll('p')
        
        content_list = [p.get_text(strip=True) for p in content]
        
        # Return the processed list or a default message
        return content_list if content_list else ["No content found"]
    except requests.exceptions.RequestException as e:
        return [f"Request error: {e}"]
    except Exception as e:
        return [f"Error: {e}"]

# Apply the function to the 'links' column and create a new column 'content'
df['content'] = df['link'].apply(content_extract)
